In [ ]:
from sklearn.datasets import fetch_20newsgroups
groups = fetch_20newsgroups()

In [ ]:
data_train = fetch_20newsgroups(subset='train', random_state=21)
train_label = data_train.target
data_test = fetch_20newsgroups(subset='test', random_state=21)
test_label = data_test.target
len(data_train.data), len(data_test.data), len(test_label)

In [ ]:
import numpy as np
np.unique(test_label)


In [ ]:
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names

all_names = names.words()
WNL = WordNetLemmatizer()
def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1 
    return(list(cleaned.values()))



In [ ]:
x_train = clean(data_train.data)
x_train[0]

In [ ]:
len(x_train)

In [ ]:
x_test = clean(data_test.data)
len(x_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(stop_words='english', max_features=1000)
X_train = tf.fit_transform(x_train)
X_test = tf.transform(x_test)
X_train.shape, X_test.shape

# SVC

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc_lib = SVC(kernel = 'linear')
parameters = {'C' : (0.5,1.0,10,100)}

In [ ]:
from sklearn.model_selection import GridSearchCV
import timeit

grid_search1 =GridSearchCV(svc_lib, parameters, n_jobs = -1, cv = 3)
start_time = timeit.default_timer()
grid_search1.fit(X_train, train_label)
final = timeit.default_timer()-start_time
print("Execution Time : ",final)

In [ ]:
print(grid_search1.best_params_)
print(grid_search1.best_score_)

In [ ]:
grid_search_best1 = grid_search1.best_estimator_
accur1 = grid_search_best1.score(X_test, test_label)
print(accur1)

# Linear SVC

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
import timeit

linear_svc = LinearSVC()
parameters = {'C': (0.5, 1, 10,100)}

grid_search2 =GridSearchCV(linear_svc, parameters, n_jobs = -1, cv = 3)
start_time = timeit.default_timer()
grid_search2.fit(X_train, train_label)
final = timeit.default_timer()-start_time
print("Execution Time : ",final)

In [ ]:
print(grid_search2.best_params_)
print(grid_search2.best_score_)

In [ ]:
grid_search_best2 = grid_search2.best_estimator_
accur2 = grid_search_best2.score(X_test, test_label)
accur2

# Model Tuning -> Linear SVC

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tf_id', TfidfVectorizer(stop_words = "english")), ('svm_im', LinearSVC())])
pipeline

parameter = {'tf_id__max_features' : (100,1000, 2000, 8000),
             'tf_id__max_df' : (0.25, 0.5),
             'tf_id__smooth_idf' : (True, False),
             'tf_id__sublinear_tf' : (True, False)
}

In [ ]:
grid_search = GridSearchCV(pipeline, parameter,cv = 3)
grid_search.fit(x_train, train_label)

In [ ]:
print(grid_search.best_params_)

In [ ]:
print(grid_search.best_score_)

In [ ]:
grid_search_acc = grid_search.best_estimator_
acc = grid_search_acc.score(x_test, test_label)
print(acc)

## Model Tuning -> SVC

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tf_id', TfidfVectorizer(stop_words = "english")), ('svm_im', SVC())])
pipeline

parameter = {'tf_id__max_features' : (100,1000, 2000, 8000),
             'tf_id__max_df' : (0.25, 0.5),
             'tf_id__smooth_idf' : (True, False),
             'tf_id__sublinear_tf' : (True, False)
}

In [ ]:
grid_search = GridSearchCV(pipeline, parameter,cv = 3)
grid_search.fit(x_train, train_label)

In [ ]:
print(grid_search.best_params_)

In [ ]:
print(grid_search.best_score_)

In [ ]:
grid_search_acc = grid_search.best_estimator_
acc = grid_search_acc.score(x_test, test_label)
print(acc)